# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner and combine it with our other data skills. In this lab you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer follow-up questions. 


### Outline:

1. Determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create Python functions to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results.

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


## Part I - Set up the DB

Start by reading SQL questions above to get an understanding of the data you will need. Then, read the documentation of Yelp API to understand what data you will receive in the response.  


Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

In [23]:
## Connect to DB server on AWS
import mysql.connector
from mysql.connector import errorcode
import time
import pandas as pd
import config
config.host

'yelppp.cdhanbzj0dic.us-east-2.rds.amazonaws.com'

In [16]:
## Create new DB
cnx = mysql.connector .connect(
    host = config1.host,
    user = config1.user,
    passwd = config1.password
    db_name = 'yelp4')
cursor = cnx.cursor()
print(cnx)

In [17]:
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

Database yelp2 does not exists.
Database yelp2 created successfully.


In [22]:
# Create a table for the businesses
cursor.execute("""CREATE TABLE IF NOT EXISTS businesses(
                                                        business_id VARCHAR(50) PRIMARY KEY,
                                                        alias TEXT,
                                                        name TEXT, 
                                                        review_count INTEGER,
                                                        rating FLOAT,
                                                        price TEXT)""")

In [ ]:
cnx.commit()

In [2]:
# Create a table for the reviews
cursor.execute("""CREATE TABLE IF NOT EXISTS reviews2(
                                                      review_id VARCHAR(50) NOT NULL PRIMARY KEY,
                                                      rating FLOAT,
                                                      time_created DATETIME,
                                                      business_id VARCHAR(50),
                                                      FOREIGN KEY fk_busi(business_id)
                                                      REFERENCES businesses(business_id))""")

In [ ]:
cnx.commit()

In [ ]:
# def drop_everything():
#     cursor.execute("""DROP TABLE businesses;""")
#     cursor.execute("""DROP TABLE reviews2;""")
#     cnx.commit()

## Part 2: Create ETL pipeline for the business data from the API

In [ ]:
# Data was saved in a json file.

# mpimport json
# iort requests
# f = open('data.json')
# data = json.load(f)

In [ ]:
term = 'italian'
location = 'New York NY'
url_params = {'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50}

In [ ]:
# write a function to make a call to the API
api_key = 'apC6W5V_e5MB6oVHzSFz7CRmKSaVKBDWlNDWrIhZDwwYHOyUs5abXwKLmxSf6M1CExNI4fYPLBKq6xVhuTS0TGygAypfTT9feHgDbi4xnOY3VTb3Vt1LySr1qfY5XXYx'

def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

In [ ]:
data_italian = yelp_call(url_params, api_key)

In [ ]:
# write a function to parse the API response so that you can easily insert the data in to the DB
def all_results(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    results = []
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results.extend(yelp_call(url_params, api_key))
        # Optional, add to DB instead

        # results_50 = yelp_call(url_params, api_key)
        # for business in results_50:
        # dynamic_data_entry(business)
        # (use dynamic insert function here)
        time.sleep(1) # wait a second
        cur += 50
    return results

In [ ]:
data_italian_all = all_results(url_params, api_key)

In [1]:
# Write a function to take your parsed data and insert it into the DB
def dynamic_data_entry(restaurant):
    business_id = restaurant['id']
    alias = restaurant['alias']
    name = restaurant['name']
    review_count = restaurant['review_count']
    rating = restaurant['rating']
    if 'price' in restaurant.keys():
        price = restaurant['price']
    else:
        price = ''
    
    cursor = cnx.cursor()
    cursor.execute("""INSERT INTO businesses (business_id, alias, name, review_count, rating, price)
                           VALUES (%s, %s, %s, %s, %s, %s)""",
                                  (business_id, alias, name, review_count, rating, price))
    cnx.commit()
    
for restaurant in data_italian_all:
    dynamic_data_entry(restaurant)

## Part 3: Create ETL pipeline for the review data from the API

In [ ]:
# write a query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant
business_ids = []
for business in data_italian_all:
    business_ids.append(business['id'])

In [ ]:
# write a function that take a business id and makes a call to the API for reviews
def yelp_call_reviews(url_params, api_key, id_b):
    url = f'https://api.yelp.com/v3/businesses/{id_b}/reviews'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    if 'reviews' in response.json().keys():
        data = response.json()['reviews']
    else:
        data = ''
    return data

In [ ]:
# parse out the relevant information
def all_results_reviews(url_params, api_key, id_b_list):
    results = []
    for id_b in id_b_list:
        url = f'https://api.yelp.com/v3/businesses/{id_b}/reviews'
        headers = {'Authorization': 'Bearer {}'.format(api_key)}
        response = requests.get(url, headers=headers, params=url_params)
        list_of_dic = yelp_call_reviews(url_params, api_key, id_b)
        for dic in list_of_dic:
            dic['business_id'] = id_b
        # insert into db
        results.extend(list_of_dic)
        time.sleep(1) # wait a second
    return results

In [ ]:
all_reviews_italian = all_results_reviews(url_params, api_key, business_ids)

In [ ]:
# with open('all_reviews_italian.json', 'w') as fp:
#     json.dump(all_reviews_italian, fp)
# with open('all_reviews_italian.json', 'w') as fp:
#     reviews = json.load(all_reviews_italian, fp)

In [ ]:
# write a function to insert the parsed data into the reviews table
def dynamic_data_entry_reviews(review):
    rating_id = review['id']
    rating = review['rating']
    time_created = review['time_created']
    business_id = review['business_id']
    
    cursor = cnx.cursor()
    cursor.execute("""INSERT INTO reviews2 (review_id, rating, time_created, business_id) 
                          VALUES (%s, %s, %s, %s)""", (rating_id, rating, time_created, business_id))
    cnx.commit()

In [ ]:
for review in all_reviews_italian:
    dynamic_data_entry_reviews(review)

## Part 4: Write SQL queries that will answer the questions posed. 

In [ ]:
# What are the 5 businesses with the highest average ratings?
cursor.execute("""SELECT name, rating 
                                      FROM businesses
                                      ORDER BY 1 DESC
                                      LIMIT 5""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

In [ ]:
# What are the 5 businesses with the lowest average ratings?
cursor.execute("""SELECT name, rating 
                                      FROM businesses
                                      ORDER BY 1
                                      LIMIT 5""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

In [ ]:
# What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs?
cursor.execute("""SELECT name, rating
                                      FROM  businesses
                                      WHERE LENGTH(price) = 1""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

In [ ]:
# How many businesses have a rating above 4.5?
cursor.execute("""SELECT COUNT(business_id)
                                            FROM  businesses
                                            WHERE rating > 4.5""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

In [ ]:
# How many businesses have a rating below 3?
cursor.execute("""SELECT COUNT(business_id)
                                            FROM  businesses
                                            WHERE rating < 3""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

In [ ]:
# Return the text of the oldest review in the table.


In [ ]:
# Return the overall rating of the business with the oldest review.


In [ ]:
# Find the highest rated business and return text of the newest review of the three.


In [ ]:
# Find the lowest rated business and return text of the newest review of the three.


###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter.***

In [ ]:
# Your code here; use a function or loop to retrieve all the results from your original request
import time

def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

def all_results(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    results = []
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results.append(yelp_call(url_params, api_key)) #Optional, add to DB instead
        time.sleep(1) #Wait a second
        cur += 50
    return results

term = 'pizza'
location = 'Astoria NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }
df = all_results(url_params, api_key)
print(len(df))
df.head()

### Sample SQL Query 

Below is a SQL query to create a table.  Additionally here is a link to create a table with a foreign key.

http://www.mysqltutorial.org/mysql-foreign-key/

```CREATE TABLE IF NOT EXISTS tasks (
    task_id VARCHAR(16) AUTO_INCREMENT,
    title VARCHAR(255) NOT NULL,
    start_date DATE,
    due_date DATE,
    status TINYINT NOT NULL,
    priority TINYINT NOT NULL,
    description TEXT,
    PRIMARY KEY (task_id)
)  ENGINE=INNODB;```